In [2]:
import pandas as pd
import gensim

In [3]:
import numpy as np
import cleantext

In [4]:
import re 
import contractions

In [30]:
from nltk.corpus import stopwords

In [40]:
df = pd.read_csv('train.csv')

In [41]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [42]:
df.shape

(7613, 5)

In [43]:
df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [44]:
 df.fillna('missing',inplace=True)

In [45]:
df.isnull().sum()

id          0
keyword     0
location    0
text        0
target      0
dtype: int64

In [13]:
%%timeit
df['target'].value_counts()

547 µs ± 48.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Text Cleaning 

In [46]:
df['text'][3]

'13,000 people receive #wildfires evacuation orders in California '

In [49]:
def clean_text(text):
    # Handling contractions
    text = contractions.fix(text)
    # Clean text inbuilt function
    text = cleantext.clean(text, no_digits=True,replace_with_digit='',no_punct=True,no_currency_symbols=True)
    # Remove # values
    text = re.sub(r'#\w+','',text)
    text = text.split(' ')
    text = [word for word in text if word not in stopwords.words('english')]
    text = ' '.join(text)
    return text

In [51]:
df['text'] = df['text'].apply(lambda x: clean_text(x))

In [52]:
df['text'][3]

'people receive wildfires evacuation orders california'

## Creating word embedding for each sentence

In [ ]:
# # One time run command 
# !python -m spacy download en

In [ ]:
# # One time run command 
# !python -m spacy download en_vectors_web_lg

In [18]:
import spacy

nlp = spacy.load('en_vectors_web_lg')
total_vectors = len(nlp.vocab.vectors)

print('Total word vectors:', total_vectors)

Total word vectors: 1070971


In [19]:
# Embedding Length 
nlp.vocab.vectors_length

300

In [53]:
def sentence_embedding(text):
    word_list = list(set(text.split(' ')))
    word_vector = nlp(word_list[0]).vector
    n = len(word_list)
    for word in word_list[1:]:
        word_vector += np.array(nlp(word).vector)
    final_vector = word_vector/n
    return np.array(final_vector)

In [54]:
l = []
for i in df['text']:
    l.append(sentence_embedding(i))

In [55]:
len(l)

7613

In [56]:
word_embedding_df = pd.DataFrame(l)


In [57]:
word_embedding_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.462142,0.052778,-0.147526,-0.022433,-0.065914,-0.163390,-0.084180,0.149339,0.007110,2.059819,...,-0.315468,0.186460,-0.325243,-0.014464,0.361223,-0.134999,0.008728,0.176468,-0.029409,-0.144857
1,-0.014817,-0.092123,-0.107169,-0.338657,0.322136,0.048467,0.321796,-0.062403,-0.188868,1.100879,...,-0.000271,0.066260,0.135487,0.009727,0.243807,0.165720,-0.062201,-0.115065,-0.260096,0.124245
2,0.046859,0.066338,-0.020495,-0.047279,-0.174517,-0.160648,-0.019068,0.017220,-0.016927,2.809466,...,-0.229352,0.030798,0.291578,0.026442,-0.054712,-0.200036,-0.222045,0.130568,0.184944,-0.079101
3,-0.226582,0.198884,-0.027298,-0.249695,-0.169410,-0.059459,-0.052042,0.202244,0.039382,2.036350,...,-0.284480,0.263769,0.177242,-0.038433,-0.046832,-0.211249,-0.077434,-0.031381,0.162820,-0.051293
4,-0.158721,-0.086279,0.112909,-0.068443,-0.019881,0.102134,-0.081687,-0.034499,-0.050314,1.590501,...,-0.045970,0.211298,0.146740,0.180153,0.105765,-0.078480,-0.094092,-0.133159,-0.052387,0.109360


In [58]:
word_embedding_df.to_csv('word_embedding_train_file.csv',index=False)